In [ ]:
import sys
sys.path.append('./campy/')
sys.path.append('./leap_rigs')

In [ ]:

import campy.campy as campy
import leap_rigs


# From campy.py
import numpy as np
import os
import time
import sys
import threading, queue
from collections import deque
import multiprocessing as mp
from campy import CampyParams
from campy.writer import campipe
from campy.display import display
from campy.cameras import unicam
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
import ast
import yaml
import logging

# From pilot_expt.py
import leap_rigs
import datetime
import glob
import h5py


In [ ]:
print("?")

In [ ]:
params = campy.LoadConfig("campy/configs/config_1cam_cpu.yaml")
params = campy.LoadSystemsAndDevices(params)

model_paths = [
    "leap_rigs/models/centroids.200823_193403.UNet.zip",
    "leap_rigs/wt_gold.13pt.multiclass_topdown.zip",
]

expt_name = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
data_path = f"./trial.{expt_name}.h5"

metadata = {"title": expt_name}

In [ ]:
if params["ffmpegPath"]:
    os.environ["IMAGEIO_FFMPEG_EXE"] = params["ffmpegPath"]
while True:
        try:
            if sys.platform == "win32":
                pool = mp.Pool(processes=params["numCams"])
                pool.map(campy.AcquireOneCamera, range(0, params["numCams"]))
            elif sys.platform == "linux" or sys.platform == "linux2":
                ctx = mp.get_context("spawn")  # for linux compatibility
                pool = ctx.Pool(processes=params["numCams"])
                p = pool.map_async(campy.AcquireOneCamera, range(0, params["numCams"]))
                p.get()

        except KeyboardInterrupt:
            print("Keyboard interrupted")
        break

campy.CloseSystems(params)

In [ ]:
params